In this notebook I intend to import the annotated data and establish a baseline performance of the non-fine-tuned davinci-instruct model <br>
prerequistes:
- A wroking prompt
- PoC that davinci works better than the other models<br><br>
Steps after:
- finetune model

next up: now that I've got a reasonable prompt for the large models, lets compare against the small ones and write up a report for thales - by tonight

In [7]:
import pandas as pd
import pickle as pkl
import openai
from sklearn.metrics import confusion_matrix
import time
import json

# import large file
with open('annotated_sample.pkl', 'rb') as f:
    pre_df = pkl.load(f)

# shorten to not murder my computer
# df = pre_df.iloc[:50]
#del pre_df

# retrieve API key
import os
api_key = os.environ.get('OPENAI_API_KEY')
# print(api_key)

In [2]:
# only execute for small experiments

# sample 30 ads, 30 non-ads
df = pd.concat([pre_df[pre_df.loc[:,"label"]==True].sample(30),
                pre_df[pre_df.loc[:,"label"]==False].sample(30)]).copy()
df.reset_index(drop=True,inplace=True)
del pre_df

In [55]:
# Generate classifications for davinci only

completions ={"0":[],"1":[]} #{"text-ada-001":[], "text-babbage-001":[], "text-curie-001":[], "text-davinci-002":[]}
i=0
for x in range(1):#,"text-ada-001", "text-babbage-001", "text-curie-001"]:
    #i=0
    for txt in df.loc[:,"caption"]:
        i+=1
        if i%10==0: print(f"Counter at {i}")

        # timer to stay within my quota
        if i%25==24:
            print(f"sleep at {i}") 
            time.sleep(65)
        
        prompt = f"Might the following post have been sponsored? \n\n Post: 'All my teenage girl dreams come true tonight! Thank you so much @sherrihill for having me perform at New York Fashion week! 🖤#SherriHillNYFW'\n Potentially Sponsored (True/False): True\n\n Post: 'Life imitating #art 🎨🎨\nlive your #fantasy ✨✨\nStrolling in #beverlyhills #losangeles \n#Illustration by @donna_adi 🎨🎨\nHappy to be back In #usa #la'\n Potentially Sponsored (True/False): False\n\n Post:{txt}\n Potentially Sponsored (True/False): "

        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            max_tokens=3,
            n=1,
            stop=None,
            temperature=0,
        )

        # Save the response text
        completions[str(x)].append(response["choices"][0]["text"])
    print(str(x)," done")

Counter at 10
Counter at 20
sleep at 24
Counter at 30
Counter at 40
sleep at 49
Counter at 50
Counter at 60
Counter at 70
sleep at 74
Counter at 80
Counter at 90
sleep at 99
Counter at 100
Counter at 110
Counter at 120
sleep at 124
Counter at 130
Counter at 140
sleep at 149
Counter at 150
Counter at 160
Counter at 170
sleep at 174
Counter at 180
Counter at 190
sleep at 199
Counter at 200
Counter at 210
Counter at 220
sleep at 224
Counter at 230
Counter at 240
sleep at 249
Counter at 250
Counter at 260
Counter at 270
sleep at 274
Counter at 280
Counter at 290
sleep at 299
Counter at 300
Counter at 310
Counter at 320
sleep at 324
Counter at 330
Counter at 340
sleep at 349
Counter at 350
Counter at 360
Counter at 370
sleep at 374
Counter at 380
Counter at 390
sleep at 399
Counter at 400
Counter at 410
Counter at 420
sleep at 424
Counter at 430
Counter at 440
sleep at 449
Counter at 450
Counter at 460
Counter at 470
sleep at 474
Counter at 480
Counter at 490
sleep at 499
Counter at 500
Cou

RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-cq5XYRxHaY8l2D7xud69qR86 on requests per min. Limit: 60.000000 / min. Current: 70.000000 / min. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.

In [57]:
# Make predictions to booleans
completions_as_boolean = {}
for model in ["0"]:
    completions_as_boolean[model] = [True if response.__contains__("rue") 
                          else False if response.__contains__("als")
                          else "warning" for response in completions[model]]

    # check for bad completions
    if "warning" in completions[model]: print("Warning")

    # obtain confusion matrix

    print(confusion_matrix(df.iloc[:1046,39],completions_as_boolean[model]))

[[216 133]
 [ 33 664]]


In [82]:
# add predictions to df
df_baseline_predictions = df.iloc[:1046].copy()
df_baseline_predictions.reset_index(drop=False, inplace=True)
df_baseline_predictions["baseline"] = completions_as_boolean["0"]

# save
with open('results.pkl', 'wb') as file:
    pkl.dump(df_baseline_predictions, file)

In [87]:
from sklearn.metrics import classification_report
print(classification_report(df_baseline_predictions["label"],df_baseline_predictions["baseline"]))

              precision    recall  f1-score   support

       False       0.87      0.62      0.72       349
        True       0.83      0.95      0.89       697

    accuracy                           0.84      1046
   macro avg       0.85      0.79      0.81      1046
weighted avg       0.84      0.84      0.83      1046



## Other models (tbd)

In [ ]:
# Generate classifications for all models

completions = {"text-ada-001":[], "text-babbage-001":[], "text-curie-001":[], "text-davinci-002":[]}
for model in ["text-davinci-003","text-ada-001", "text-babbage-001", "text-curie-001"]:
    i=0
    for txt in df.loc[:,"caption"]:
        i+=1
        if i%10==0: print(f"Counter at {i}")

        # timer to stay within my quota
        if i==24: time.sleep(65)
        if i==49: time.sleep(65)
        if i==74: time.sleep(65)
        if i==99: time.sleep(65)

        prompt1 = f"Judge whether it is likely that the following caption comes from a post that has been sponsored. If you are very uncertain err rather towards judging 'False'.\n\n Post: 'aaaa meu look de hoje é da @cea_brasil ❤️ patrocinadora e dona do look oficial do Rock in Rio! Aproveitem pra acompanhar o perfil da C&A pra ver todos os lookinhos maravilhosos! Amo muito! ❤️'\n Sponsored (True/False): True\n\n Post: 'side by side ~ sisters ❤️'\n Sponsored (True/False): True\n\n Post:{txt}\n Sponsored (True/False):"
        
        prompt2 = f"Judge whether it is likely that the following caption comes from a post that has been sponsored. If you are very uncertain err rather towards judging 'True'.\n\n Post: 'aaaa meu look de hoje é da @cea_brasil ❤️ patrocinadora e dona do look oficial do Rock in Rio! Aproveitem pra acompanhar o perfil da C&A pra ver todos os lookinhos maravilhosos! Amo muito! ❤️'\n Sponsored (True/False): True\n\n Post: 'side by side ~ sisters ❤️'\n Sponsored (True/False): True\n\n Post:{txt}\n Sponsored (True/False):"

        # Generate a response from openai.
        # Flexibly choose ada/babbage/curie/davinci as engine. For davinci, use text-davinci-002.
        prompt=("warning")
        if model in ["text-ada-001", "text-babbage-001"]:
            if i==1: print("pompt1")
            prompt=prompt1
        elif model in ["text-curie-001", "text-davinci-002"]:
            if i==1: print("pompt2")
            prompt=prompt2
        else:
            print("warning")
        response = openai.Completion.create(
            engine=model,
            prompt=prompt,
            max_tokens=5,
            n=3,
            stop=None,
            temperature=0,
        )

        # Save the response text
        completions[model].append(response["choices"][0]["text"])
    print(model+" done")

In [5]:
with open('results.pkl', 'rb') as f:
    results = pkl.load(f)

In [6]:
results

,index,username,caption,caption_hashtags,tagged_users,tagged_users_full,likes,comments,is_ad,caption_is_edited,...,dt_year_mon,caption_text_symbols,has_sponsored_keywords,has_disclosures,keyword_tfidf,keyword_keybert,captions_task,caption_task,label,baseline
0,87029,weworewhat,"I can’t believe it’s May. For me, that usually...",[],"[fendi, fandhjewellery, cosstores, zara, vehla...","[Fendi, F + H, COS, ZARA Official, VEHLA EYEWEAR]",17112,131,False,False,...,2020/05,i can t believe it s may for me that usually b...,False,False,"[woah is this, different right, of our near, n...","[feeling summer starting, 28th feeling summer,...","I can’t believe it’s May. For me, that usually...","I can’t believe it’s May. For me, that usually...",False,False
1,87072,weworewhat,You guys know I’m obsessed with blue light gla...,"[#thedewers, #ad]",[goodhabitskin],[Goodhabit],11169,85,False,True,...,2020/04,you guys know i m obsessed with blue light gla...,True,True,"[blue light, blue, light, it protects against,...","[exposure blue light, obsessed blue light, exp...",You guys know I’m obsessed with blue light gla...,You guys know I’m obsessed with blue light gla...,True,True
2,87105,weworewhat,Moving my body during this time has not only h...,"[#fpmovement, #ad]",[],[],27161,259,False,True,...,2020/04,moving my body during this time has not only h...,True,True,"[workouts, days that we, kept us active, activ...","[live workouts, free live workouts, workouts g...",Moving my body during this time has not only h...,Moving my body during this time has not only h...,True,True
3,87181,weworewhat,new @shopweworewhat coverup in a print inspire...,"[#iwantklarna, #klarnapartner, #ad]",[klarna.usa],[Klarna US],18029,305,False,False,...,2020/02,new @shopweworewhat coverup in a print inspire...,True,True,"[vintage rug forget, by my bedroom, new coveru...","[new coverup print, rug forget shop, new cover...",new @shopweworewhat coverup in a print inspire...,new @shopweworewhat coverup in a print inspire...,True,True
4,87196,weworewhat,Another rainy day but my curated @gilt sale en...,"[#gotitongilt, #ad]",[gilt],[Gilt.com],19554,101,False,False,...,2020/02,another rainy day but my curated @gilt sale en...,True,True,"[curated sale ends, added few more, more piece...","[curated sale ends, rainy day curated, day cur...",Another rainy day but my curated @gilt sale en...,Another rainy day but my curated @gilt sale en...,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,362673,jarrylee,carpe noctem (photographed by Joseph Franciosa...,[],"[gothamfotografia, deeptimua, jarrylee]","[gothamfotografia, Deepti Sheladia✨, JARRY LEE]",6324,994,False,False,...,2019/01,carpe noctem photographed by joseph franciosa ...,False,False,"[sheladia, noctem photographed, makeup by deep...","[carpe noctem photographed, noctem photographe...",carpe noctem (photographed by Joseph Franciosa...,carpe noctem (photographed by Joseph Franciosa...,False,False
1042,362679,jarrylee,honored to be named @ModelMayhem’s Model of th...,[],"[jarrylee, modelmayhem, uga_talent, modelmayhe...","[JARRY LEE, Model Mayhem, UGA Talent Agency, M...",6233,743,False,False,...,2019/01,honored to be named @modelmayhem s model of th...,False,False,"[named model of, for january 2019, named model...","[model month january, honored named model, nam...",honored to be named @ModelMayhem’s Model of th...,honored to be named @ModelMayhem’s Model of th...,False,True
1043,363199,jacobmanningyoga,My leg routine takes me about 1 to 4 hours. Ha...,[],"[gymshark, gymsharktrain]","[Homeshark, Homeshark Train]",3976,73,False,True,...,2020/04,my leg routine takes me about to hours having ...,False,False,"[foam role, and, ankles, foam, my]","[leg routine takes, healthy legs takes, leg ro...",My leg routine takes me about 1 to 4 hours. Ha...,My leg routine takes me about 1 to 4 hours. Ha...,True,True
1044,363268,jacobmanningyoga,I'm totally going to talk about god right now....

In [10]:
results[(results["has_disclosures"]==True) & (results["label"]==False)].iloc[0,-3]

'❗️GIVEAWAY CLOSED❗️GIFTCARD GIVEAWAY❗️ We’ve teamed up with @PDQFreshFood to give one of you a $25 Gift Card! 💰 We honestly loved this spot when we devoured here recently! Delicious Fried Chicken, Cheesy Tater Tots, Salads, etc. 100% recommended. #DEVOURPOWER #DEVOURPOWERNewJersey - - - TO ENTER: 1️⃣: Follow @PDQFreshFood & @DEVOURPOWER 2️⃣: Tag a friend in the comments below! - - - Winner will be announced and sent a DM in 24 hours! Good luck! 😁'